This notebook uses the network created by ```Topology_clique_activity_experiment.ipynb```, and ablates all connections to run a reference unconnected run.

In [1]:
import os
os.environ["SNUDDA_DATA"] = "../../../../BasalGangliaData/data/"
assert os.path.isdir(os.getenv("SNUDDA_DATA")), f"You need to have BasalGangliaData installed for this example."

network_path = "topology100"
network_file = os.path.join(network_path, "network-synapses.hdf5")
simplex_file_name = os.path.join(network_path, "topology100_D6_simplices0.simplices")
ablated_network_file = os.path.join(network_path, "network-fully-ablated.hdf5")

remove_sink_to_source_connections = True

In [2]:
from snudda.analyse.analyse_topology import SnuddaAnalyseTopology
sat = SnuddaAnalyseTopology(network_file=network_file)
sat.load_simplex_file(simplex_file_name=simplex_file_name)
mult = sat.get_fixed_multiplicity()

Loading 99 neurons with 6999 synapses and 0 gap junctions
Loading 99 neurons with 6999 synapses and 0 gap junctions
Loaded simplex data of dimension 6 from topology100/topology100_D6_simplices0.simplices


Here we list all simplices with 1 FS and 4 ISPN, then we pick one of them as the simplex to analyse.

In [3]:
filt_mult = sat.filter_multiplicity(mult, dimension=6, neuron_type_list=[["FS", 1], ["iSPN", 4]],
                       multiplicity_requirement=None)

In [4]:
selected_clique_neurons = list(filt_mult)[0]

In [5]:
selected_clique_neurons

(92, 34, 43, 82, 93, 95, 71)

### Ablating network

In [6]:
from snudda.utils.ablate_network import SnuddaAblateNetwork
san = SnuddaAblateNetwork(network_file=network_file)
san.only_keep_neuron_id(selected_clique_neurons)

Loading 99 neurons with 6999 synapses and 0 gap junctions


The cell below removes sink to source connections if the ```remove_sink_to_source``` flag is set.

In [7]:
san.ablate_all_synapses()

Note that the neurons get remapped, when we ablate the network and remove a subset of the neurons.

In [8]:
san.write_network(out_file_name=ablated_network_file, print_remapping=True)

Writing to topology100/network-fully-ablated.hdf5
Copying morphologies
Keeping 7 neurons.

Remapping neurons:
34 -> 0
71 -> 1
43 -> 2
82 -> 3
92 -> 4
93 -> 5
95 -> 6

0/0 synapses processed
Filtering done.
Copying synapses and gap junctions
6999 / 0 synapse rows parsed
Synapse matrix written.
Keeping 0 synapses (out of 6999)
0 / 0 gap junction rows parsed
Gap junction matrix written.
Keeping 0  gap junctions (out of 0)


### Simulating network

Next step is running the network with current injections (we should also run it with synaptic input).

#### NEED TO CHANGE TO BasalGangliaData

In [ ]:
experiment_config_file = "topology_clique_simulation_experiment-6a.json"
from snudda.simulate.pair_recording import PairRecording 
pr = PairRecording(network_path=network_path, network_file=ablated_network_file,
                   experiment_config_file=experiment_config_file)

Pair recording output file: topology100/simulation/TCS-output-6a.hdf5
Loading 7 neurons with 0 synapses and 0 gap junctions


In [ ]:
pr.run()

In [ ]:
%matplotlib notebook
pr.plot_trace_overview()

In [ ]:
pr.plot_traces()